In [ ]:
#connect drive for files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#enter the path to directory
%cd /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts

/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/nlp


In [ ]:
#install all requirements
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/Shareddrives/ECE%20226%20Project/archai-neurips-lts/archai-neurips-lts
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/NVIDIA/dllogger.git to /tmp/pip-install-li_6293x/dllogger_cceffc71d78045c4af2df785c1f0709d
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/dllogger.git /tmp/pip-install-li_6293x/dllogger_cceffc71d78045c4af2df785c1f0709d
  Resolved https://github.com/NVIDIA/dllogger.git to commit 0540a43971f4a8a16693a9de9de73c1072020769
  Preparing metadata (setup.py) ... done
  Attempting uninstall: archai
    Found existing installation: archai 0.6.0
    Uninstalling archai-0.6.0:
      Successfully uninstalled archai-0.6.0
  Running setup.py develop for archai


In [ ]:
#run search for opt with param limits
!python archai/nlp/search.py --n_iter 30 --model_type hf_ctrl --n_layer 2 3 4 5 6 7 8 --n_head 2 4 8 --div_val 4 --param_constraint_lower 25000 --param_constraint_upper 10000000 --device cuda --device_name titanxp


baseline - finding the baseline architecture latency memory and decoder params

In [ ]:
!python archai/nlp/search.py --profile_baseline --model_type hf_ctrl --n_layer 2 3 4 5 6 7 8 --n_head 2 4 8 --div_val 4 --param_constraint_lower 25000 --param_constraint_upper 10000000 --device cuda --device_name titanxp --dataset wt2

2023-06-16 02:16:10.510228: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
STAGE:2023-06-16 02:16:18 3545:3545 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-06-16 02:16:18 3545:3545 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-06-16 02:16:18 3545:3545 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
config_0_j1 has: 
            396288 decoder_params
            2956288 total_params
            0.0026s latency
            57.9805MB memory
STAGE:2023-06-16 02:16:19 3545:3545 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-06-16 02:16:19 3545:3545 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-06-16 02:16:19 3545:3545 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
config_0_j2 has: 
            792064 decoder_params
            3352064 total_params
            0.0047s latency
            75.38

select pareto which are closest to the baseline

In [ ]:
!python archai/nlp/search.py --select_pareto --model_type hf_ctrl --n_layer 2 3 4 5 6 7 8 --n_head 2 4 8 --div_val 4 --param_constraint_lower 25000 --param_constraint_upper 10000000 --device cuda --device_name titanxp --dataset wt2

2023-06-16 02:17:18.571970: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Selected 17 of pareto jobs based on their proximity to the baseline architectures.
Unique Pareto-frontier jobs: 17


Training pareto architectures (will take time) change the path here. once trained, move the generated files in logdir to logdir/hf_ctrl_titanxp_3d/pareto/

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j0  --n_layer 2 --d_model 192 --d_inner 2560 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:21:40.417439: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j0
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j0', experiment_name='config_0_j0', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j1  --n_layer 2 --d_model 128 --d_inner 3648 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:23:00.135007: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j1
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j1', experiment_name='config_0_j1', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j2  --n_layer 2 --d_model 128 --d_inner 256 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:24:10.022321: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j2
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j2', experiment_name='config_0_j2', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j3  --n_layer 2 --d_model 128 --d_inner 1216 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:25:09.184602: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j3
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j3', experiment_name='config_0_j3', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j4  --n_layer 2 --d_model 256 --d_inner 2176 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:26:12.571121: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j4
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j4', experiment_name='config_0_j4', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j5  --n_layer 2 --d_model 384 --d_inner 2240 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:27:39.479743: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j5
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j5', experiment_name='config_0_j5', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j6  --n_layer 2 --d_model 256 --d_inner 4032 --n_head 8 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:29:28.215279: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j6
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j6', experiment_name='config_0_j6', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j7  --n_layer 2 --d_model 192 --d_inner 1216 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:31:04.968079: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j7
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j7', experiment_name='config_0_j7', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j8  --n_layer 2 --d_model 256 --d_inner 2560 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:32:21.733036: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j8
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j8', experiment_name='config_0_j8', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j9  --n_layer 2 --d_model 512 --d_inner 3840 --n_head 8 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:33:49.872637: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j9
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j9', experiment_name='config_0_j9', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j10  --n_layer 2 --d_model 128 --d_inner 832 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:36:13.035323: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j10
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j10', experiment_name='config_0_j10', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j11  --n_layer 2 --d_model 256 --d_inner 2112 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:37:15.463038: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j11
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j11', experiment_name='config_0_j11', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j12  --n_layer 2 --d_model 128 --d_inner 2688 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:38:39.222927: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j12
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j12', experiment_name='config_0_j12', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j13  --n_layer 2 --d_model 256 --d_inner 2688 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:39:44.652189: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j13
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j13', experiment_name='config_0_j13', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j14  --n_layer 2 --d_model 512 --d_inner 3840 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:41:10.931239: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j14
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j14', experiment_name='config_0_j14', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j15  --n_layer 2 --d_model 128 --d_inner 1792 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:43:32.852609: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j15
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j15', experiment_name='config_0_j15', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j16  --n_layer 2 --d_model 128 --d_inner 1344 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:44:35.500503: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j16
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j16', experiment_name='config_0_j16', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

Training baselines ( will take time). Once training is done, move the files from logdir to baseline_logs/hf_ctrl/wt2/ folder

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j1  --n_layer 1 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:51:19.985998: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j1
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j1', experiment_name='config_0_j1', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j2  --n_layer 2 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:49:45.643622: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j2
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j2', experiment_name='config_0_j2', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j3  --n_layer 3 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:52:32.015619: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j3
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j3', experiment_name='config_0_j3', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j4  --n_layer 4 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:53:54.091744: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j4
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j4', experiment_name='config_0_j4', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j5  --n_layer 5 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:55:20.484014: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j5
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j5', experiment_name='config_0_j5', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j6  --n_layer 6 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:56:53.291094: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j6
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j6', experiment_name='config_0_j6', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j7  --n_layer 7 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 02:58:29.344450: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j7
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j7', experiment_name='config_0_j7', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_ctrl --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j8  --n_layer 8 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 03:00:12.531967: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j8
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j8', experiment_name='config_0_j8', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

Plot pareto baseline

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/search.py --plot_pareto_baseline --n_layer 2 3 4 5 6 7 8 --div_val 4 --n_head 2 4 8 --param_constraint_lower 25000 --param_constraint_upper 2500000 --device cuda --device_name titanxp --dataset wt2 --model_type hf_ctrl

2023-06-16 03:03:39.327980: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
{'config_0_j2_train_log.json': {'train_elapsed': 60.02585816383361, 'valid_ppl': 647.0923755153102, 'amlt_job': None}, 'config_0_j1_train_log.json': {'train_elapsed': 57.088120460510254, 'valid_ppl': 678.5858208598986, 'amlt_job': None}, 'config_0_j3_train_log.json': {'train_elapsed': 65.24399042129517, 'valid_ppl': 623.1823770853827, 'amlt_job': None}, 'config_0_j4_train_log.json': {'train_elapsed': 69.54112458229065, 'valid_ppl': 611.2975286468773, 'amlt_job': None}, 'config_0_j5_train_log.json': {'train_elapsed': 75.06935381889343, 'valid_ppl': 591.806840263209, 'amlt_job': None}, 'config_0_j6_train_log.json': {'train_elapsed': 78.95293831825256, 'valid_ppl': 561.0152422524453, 'amlt_job': None}, 'config_0_j7_train_log.json': {'train_elapsed': 84.36257600784302, 'valid_ppl': 565.7247666332015, 'amlt_job': None}, 'config_0_j8_train_log.json': {'train_elapsed': 8

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/search.py --gen_tables --n_layer 2 3 4 5 6 7 8 --div_val 4 --n_head 2 4 8 --param_constraint_lower 25000 --param_constraint_upper 2500000 --device cuda --device_name titanxp --dataset wt2 --model_type hf_gpt2

2023-06-12 08:02:05.062996: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
table_titanxp.tex
{'vocab_size': 10000, 'n_positions': 192, 'n_embd': 128, 'd_embed': 512, 'n_inner': 256, 'resid_pdrop': 0.1, 'attn_pdrop': 0.0, 'initializer_range': 0.02, 'n_layer': 2, 'n_head': 2, 'd_head': 64, 'embd_pdrop': 0.0, 'tie_weight': True, 'activation_function': 'gelu_new', 'layer_norm_epsilon': 1e-05, 'summary_type': 'cls_index', 'summary_use_proj': True, 'summary_activation': None, 'summary_first_dropout': 0.1, 'summary_proj_to_labels': True, 'scale_attn_weights': True, 'use_cache': True, 'scale_attn_by_inverse_layer_idx': False, 'reorder_and_upcast_attn': False, 'bos_token_id': None, 'eos_token_id': None, 'return_dict': True, 'output_hidden_states': False, 'output_attentions': False, 'torchscript': False, 'torch_dtype': None, 'use_bfloat16': False, 'tf_legacy_loss': False, 'pruned_heads': {}, 'tie_word_embeddings': True, 'is_encoder_decoder': False